# 12.1 内置类型存在无法覆盖的方法

In [6]:

class DoppelDict(dict):

    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2) # 覆盖父类的__setitem__方法

In [7]:
# 初始化没有调用我们覆盖的函数，而是使用了父类的__init__方法
dd = DoppelDict({'one' : 1})
dd 

{'one': 1}

In [8]:
# 直接调用setitem方法，正常运行我们的覆盖版本
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [9]:
# 而使用update函数时，又用了内置的方法！
dd.update(three = 3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

 直接子类化内置类型(如 dict、list 或 str)容易出 错，因为内置类型的方法通常会忽略用户覆盖的方法。不要子类化 内置类型，用户自己定义的类应该继承 collections 模块 (http://docs.python.org/3/library/collections.html)中的类，例如 UserDict、UserList 和 UserString，这些类做了特殊设计， 因此易于扩展。

本节所述的问题只发生在 C 语言实现的内置类型内部的方法委托
上，而且只影响直接继承内置类型的用户自定义类。如果子类化使用 Python 编写的类，如 UserDict 或 MutableMapping，就不会受此影 响。3

# python是强类型

如果一门语言很少隐式转换类型，说明它是强类型语言；如果
经常这么做，说明它是弱类型语言。Java、C++ 和 Python 是强类型
语言。PHP、JavaScript 和 Perl 是弱类型语言。

# 多继承和方法解析顺序

In [ ]:
class A:
    def ping(self):
        print('ping:',self)

class B(A):
    def pong(self):
        print('pong', self)

class C(A):
    def pong(self):
        print('PONG', self)

class D(B,C):
    def ping(self):
        super().ping()
        print('post-ping:', self)

    def Aping(self):
        A.ping(self) # 而不是super().ping()
        # 直接在类上调用实例方法时，必须显式传入 self 参数，
        # 因为这样访问的是未绑定方法（unbound method）。
        # 建议使用super() 更安全可靠

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self) # 直接指定超类的方法

In [ ]:
d = D()
d.pong()

In [ ]:
# 超类中的方法都可以直接调用，此时要把实例作为显式参数传入。
C.pong(d)

In [ ]:
# 使用 super() 调用方法时，会遵守方法解析顺序
d.pingpong()

In [ ]:
# 类都有一个名为 __mro__ 的属性，它的值是一个元组，
# 按照方法解析顺序列出各个超类，从当前类一直向上，直到object 类。

# 方法解析顺序使用 C3 算法计算

D.__mro__

In [ ]:
d.Aping()

# 处理多重继承
1. 把接口继承和实现继承区分开

    使用多重继承时，一定要明确一开始为什么创建子类。主要原因可能有：
    - 继承接口，创建子类型，实现“是什么”关系
    - 继承实现，通过重用避免代码重复

   其实这两条经常同时出现，不过只要可能，一定要明确意图。通过
继承重用代码是实现细节，通常可以换用组合和委托模式。而接口
继承则是框架的支柱。
2. 使用抽象基类显式表示接口

    现代的 Python 中，如果类的作用是定义接口，应该明确把它定义为
抽象基类。Python 3.4 及以上的版本中，我们要创建 abc.ABC 或其
他抽象基类的子类（如果想支持较旧的 Python 版本，参见 11.7.1
节）。
3. 通过混入重用代码

    如果一个类的作用是为多个不相关的子类提供方法实现，从而实现
重用，但不体现“是什么”关系，应该把那个类明确地定义为混入类
（mixin class）。从概念上讲，混入不定义新类型，只是打包方
法，便于重用。混入类绝对不能实例化，而且具体类不能只继承混
入类。混入类应该提供某方面的特定行为，只实现少量关系非常紧
密的方法。

4. 在名称中明确指明混入

    因为在 Python 中没有把类声明为混入的正规方式，所以强烈推荐在
名称中加入 ...Mixin 后缀。Tkinter 没有采纳这个建议，如果采
纳的话，XView 会变成 XViewMixin，Pack 会变成
PackMixin，图 12-3 中所有使用 «mixin» 标记的类都应该这么
做。

05. 抽象基类可以作为混入，反过来则不成立

    抽象基类可以实现具体方法，因此也可以作为混入使用。不过，抽
象基类会定义类型，而混入做不到。此外，抽象基类可以作为其他
类的唯一基类，而混入决不能作为唯一的超类，除非继承另一个更
具体的混入——真实的代码很少这样做。

    抽象基类有个局限是混入没有的：抽象基类中实现的具体方法只能
与抽象基类及其超类中的方法协作。这表明，抽象基类中的具体方
法只是一种便利措施，因为这些方法所做的一切，用户调用抽象基
类中的其他方法也能做到。

06. 不要子类化多个具体类

    **具体类可以没有，或最多只有一个具体超类**。 也就是说，具体类
的超类中除了这一个具体超类之外，其余的都是抽象基类或混入。
例如，在下述代码中，如果 Alpha 是具体类，那么 Beta 和
Gamma 必须是抽象基类或混入：
    ```python
    class MyConcreteClass(Alpha, Beta, Gamma):
        """这是一个具体类，可以实例化。"""
        # ……更多代码……
    ```
07. 为用户提供聚合类

    如果抽象基类或混入的组合对客户代码非常有用，那就提供一个
类，使用易于理解的方式把它们结合起来。Grady Booch 把这种类
称为聚合类（aggregate class）。
    ```
    class Widget(BaseWidget, Pack, Place, Grid):
        """Internal class.
        Base class for a widget which can be positioned with the
        geometry managers Pack, Place or Grid."""
        pass
    ```
    Widget 类的定义体是空的，但是这个类提供了有用的服务：把四
个超类结合在一起，这样需要创建新小组件的用户无需记住全部混
入，也不用担心声明 class 语句时有没有遵守特定的顺序。

08. “优先使用对象组合，而不是类继承”

    优先使用组合能让设计更灵活。例如，对
tkinter.Widget 类来说，它可以不从全部几何管理器中继承方
法，而是在小组件实例中维护一个几何管理器引用，然后通过它调
用方法。毕竟，小组件“不是”几何管理器，但是可以通过委托使用
相关的服务。这样，我们可以放心添加新的几何管理器，不必担心
会触动小组件类的层次结构，也不必担心名称冲突。即便是单继
承，这个原则也能提升灵活性，因为子类化是一种紧耦合，而且较
高的继承树容易倒。

    组合和委托可以代替混入，把行为提供给不同的类，但是不能取代
接口继承去定义类型层次结构。